# School of Electrical and Information Engineering
### University of the Witwatersrand, Johannesburg
### ELEN4022 — Full Stack Quantum Computing

In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import *
# *QuantumCircuit, execute, Aer, IBMQ
# from qiskit.compiler import transpile, assemble
# # from qiskit.tools.jupyter import *
# from qiskit.visualization import *
# from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

This can be represented programmatically using arrays using Numpy. In Python, we can ecapsulate this representation as a subclass of the Qubit and it's state. 

In [ ]:
import numpy as np
import math


import abc
import math
import cmath



class QBaseState(abc.ABC):
    """Common properties of quantum states.
    This class represents an abstract method used by QBit and QGate.
    It presents a column vector and row vector representation 
    """

    @property
    def ket(self):
        return self._state

    @property
    def bra(self):
        return np.conjugate(self._state.T)

    @abc.abstractmethod
    def __init__(self, state):
#         self._state = state

It may prove useful to define a string initialization of all quantum properties, where ZERO and ONE are string representations of $|0\rangle$ and $|1\rangle$ respectively. We may represent a Qbit with a two level quantum system as a class that inherits from the above mentioned QBaseState.

In [ ]:
class QBit(QBaseState):
    """A two level quantum system.
    Attributes:
        column vector (ket) and  row vector (bra)
    Constants:
        ZERO: string representation of 0 qubit
        ONE : string representation of 1 qubit
    """
    
    ZERO = "0"
    ONE  = "1"


    def __init__(self, qubit_str):
        """initialize a QBit object from its string representation.
        Args:
            qubit_str:  "1" or "0"
        """
        if qubit_str == QBit.ZERO:
            state = np.array([[1], [0]])
        elif qubit_str == QBit.ONE:
            state = np.array([[0], [1]])

        else:raise ValueError("A qubit must be one 0 or 1")
                      
        super().__init__(state)
        self._state_str = qubit_str


We have included a variable that further encapsulates the string representation. This allows Qbit objects to be initialized using strings and conveniently by refrencing that string class variable as follows:

In [ ]:

# Initialize using a string
ZERO_STRING = QBit("0")
ONE_STRING  = QBit("1")

# Initialize using class constant variable
ZERO = QBit(QBit.ZERO)

ONE  = QBit(QBit.ONE)

The following Python class defines a Quantum Gate it's properties as well as some attributes that are defined and enforced. The complex attributes of a quantum gate matrix are enfored, the matrix is checked for unitary properties. The multiplication of gates is defined as well as the string initialization of a quantum gate object and the initialization of a quantum control gate object. 

In [ ]:
class QGate(object):
    """A general quantum gate.
    QGate is a representation of a quantum gate using complex numpy matrices.
   
    Attributes:
        matrix: the matrix representation of the gate
    """

    @property
    def matrix(self):
        """Return the matrix."""
        return self._matrix

    @property
    def dagger(self):
        """Return the Hermitian of the matrix."""
        return self._matrix.H
    

    def __init__(self, matrix, multiplier=0):
        """Create a quantum gate from a numpy matrix.
        
        Args:
            matrix: a complex typed numpy matrix           
        """

        if matrix.dtype == np.dtype('complex128'):
            self._matrix = matrix
        else:
            self._matrix = matrix.astype('complex128')

        shape = self._matrix.shape
        if shape[0] != shape[1]:
            raise ValueError("Gate is not a square matrix")

#         is_unitary = np.allclose(self._matrix.H * self._matrix, np.eye(shape[0]))
#         if not is_unitary:
#             raise ValueError("Gate is not unitary")
        
        if multiplier:
            self._matrix = self._matrix * multiplier

    def __mul__(self, other):
        """Override matrix multiplication operator on QGate objects."""
        if hasattr(other, "matrix"):
            return QGate(self.matrix * other.matrix)
        return QGate(self.matrix, other)

    
    @classmethod
    def String(cls, matrix_str, multiplier=0):
        """Create a quantum gate from a string.
        Args:
            matrix_str: the string representation of the quantum gate
        """
        matrix = np.matrix(str(matrix_str), dtype='complex128')
        return QGate(matrix, multiplier)

   
                
    @classmethod
    def Multiplication(cls, qu_gates):
        """ Create a quantum gate by multiplying existing gates
        Args:
            qu_gates: a list of QGates to multiply
        """
        if qu_gates:
            matrix = qu_gates[0].matrix
            for i in range(1, len(qu_gates)):
                if matrix.shape != qu_gates[i].matrix.shape:
                    raise ValueError("The matrices have different shapes")

                matrix = matrix * qu_gates[i].matrix

            return QGate(matrix)
        else:
            raise ValueError("No gates specified")


    @classmethod
    def Tensor(cls, qu_gates):
        """ Create a quantum gate by the tensor product of other gates
        """
        if qu_gates:
            matrix = np.matrix([1], dtype="complex128")
            for i in range(len(qu_gates) - 1, -1, -1):
                matrix = np.kron(qu_gates[i].matrix, matrix)

            return QGate(matrix)
        else:
            raise ValueError("No gates specified")

    @classmethod
    def Control(cls, qu_gate, control_qubit=1, target_qubit=2, num_qubits=2):
        """ Creates a CONTROL-U gate
        Args:
            qu_gate: a U QGate
            control_qubit: index of the control bit, the default is 1
            target_qubit: the index of the target bit, the default is 2
            num_qubits: total number of qubits, the default is 2
        """
        
        """Perform some basic logic checks"""
        if num_qubits < 2:
            raise ValueError("control gates must operate on at least 2 qubits")

        if control_qubit == target_qubit:
            raise ValueError("control qubit must be different than target qubit")

        if control_qubit > num_qubits:
            raise ValueError("control qubit cannot be greater than total number of qubits")

        if target_qubit > num_qubits:
            raise ValueErrorr("target qubit cannot be greater than total number of qubits")

        index = 1
        
        
        control_mat = 1
        target_mat = 1
        
        while index <= num_qubits:
            if index == control_qubit:
                    control_mat = np.kron(control_mat, np.eye(2))
                    target_mat = np.kron(target_mat, qu_gate.matrix)

            elif index == target_qubit:
                    control_mat = np.kron(control_mat, ZERO.ket * ZERO.bra)
                    target_mat = np.kron(target_mat, ONE.ket * ONE.bra)

            else:
                control_mat = np.kron(control_mat, np.eye(2))
                target_mat = np.kron(target_mat, np.eye(2))

            index += 1

        control_gate = control_mat + target_mat
        return QGate(control_gate)
    
    
    
    @classmethod
    def Swap(cls, a, b, num_qubits):
        """
        Initialize a swap gate between two qubits a and b
        """
        control_ab = QGate.Control(X, a, b, num_qubits)
        control_ba = QGate.Control(X, b, a, num_qubits)

        matrix = control_ab.matrix * control_ba.matrix * control_ab.matrix
        return QGate(matrix)
    
    
    @classmethod
    def Phase(cls, k):
        """
        Initialize a phase gate R(k)
        :param k: the power of the phase exp(2pi*j / 2 ** k)
        :return: the phase gate R(k)
        """
        phase = (2 * cmath.pi * 1j) / (2**k)
        phase_matrix = np.matrix([[1, 0], [0, cmath.exp(phase)]], dtype="complex128")
        return QGate(phase_matrix)


Once the quantum gate is fully defined, single-qubit gates can be intialized as follows:

In [ ]:
X = QGate.String("0 1; 1 0")
P = QGate.Phase(3)

A multi-qubit $C_{X}$ gate using the LSB convention


In [ ]:
CNOT = QGate.Control(X,1,2)
print(CNOT.matrix)

and $C_{X}$ gate using the MSB convention

In [ ]:
CNOT = QGate.Control(X,2,1)
print(CNOT.matrix)

We will now attempt to create the QFT circuit using python and Qiskit. The results will be compared for validation.

# Python Implementation

In [ ]:
ROOT2 = 1/math.sqrt(2)
print(ROOT2)
H = QGate.String("1 1; 1 -1", ROOT2)
I = QGate(np.matrix(np.eye(2)))

In [ ]:
def QFT(num_qubits):
    """Implementation of the quantum fourier transform.
    Args:
        num_qubits
    """
    h_list = [I for x in range(0, num_qubits)]
    qft_matrices = []
    for qubit in range(1, num_qubits+1):
        sub_matrices = []
        
        # create the Haddamard part
        h_list[qubit-1] =  H
        haddamard = QGate.Tensor(h_list)
        sub_matrices.insert(0, haddamard)
        h_list[qubit-1] = I

        # create the control rotations
        i = qubit + 1
        phase = 2
        while i <= num_qubits:
            phase_gate = QGate.Phase(phase)
            control = QGate.Control(phase_gate, i, qubit, num_qubits)
            sub_matrices.insert(0, control)
            i += 1
            phase += 1

        qft_matrices.insert(0, QGate.Multiplication(sub_matrices))

    for qubit in range(1, int(num_qubits / 2) + 1):
        swap = QGate.Swap(qubit, num_qubits - qubit + 1, num_qubits)
        qft_matrices.insert(0, swap)

    return QGate.Multiplication(qft_matrices)



if __name__ == "__main__":
    QFT3 = QFT(3)
    ROWS = QFT3.matrix.shape[0]
    for row in range(0, ROWS):
        row_txt = ""
        for col in range(0, ROWS):
            row_txt += "%s\t" % QFT3.matrix.item((row, col))
        print(row_txt + "\n")

## QISKIT Implementation

In [ ]:
def QftQiskit(qc, reg, n):
    """
    Computes the quantum Fourier transform of reg, one qubit at
    a time.
    Apply one Hadamard gate to the nth qubit of the quantum register reg, and 
    then apply repeated phase rotations with parameters being pi divided by 
    increasing powers of two.
    """ 
    if n == 0: # Exit function if circuit is empty
        return circuit
    
    qc.h(reg[n])    
    for i in range(0, n):
        qc.cu1(pi/float(2**(i+1)), reg[n-(i+1)], reg[n])    


In [ ]:
n = int(input("Enter number of Qubits") or "2")

a = QuantumRegister(n+1, "a") 
b = QuantumRegister(n+1, "b")     
cl = ClassicalRegister(n+1, "cl") 
qc = QuantumCircuit(a, b, cl, name="qc")

for i in range(0, n+1):
        QftQiskit(qc, a, n-i, math.pi)
       
        
for i in range(0, n+1):
    qc.measure(a[i], cl[i])
    print(qc.qasm()) 

    num_shots = 1024
    job = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shot = num_shots)
    
    qasm_result = job.result().get_counts()
    
    print(qasm_result)


counts = result.get_counts("qc")
print(counts)
#Select result with maximum probabilities
output = max(counts.items(), key=operator.itemgetter(1))[0]
print(output)